In [45]:
'''
IMPORTS
'''
import random
import numpy as np
import pandas as pd
from multiprocessing import *
import warnings
warnings.filterwarnings("ignore")
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD
import math


In [ ]:
'''
LEITURA DE DADOS
Mesma coisa do codigo do Felipe
'''

df = pd.read_excel('/home/greff/Desktop/scheduler-class-assistant/dataset/Dados_monitores.xlsx')

""" 
D - Conjunto de disciplinas
Disciplinas-Turmas: compostos das disciplinas e das turmas: string(<disciplina>-<turma>)
Ex: ['Matemática-1','SCC0524 - Cálculo 496','Física-1', 'Química-1', 'Economia-2']
"""
disciplinas = set()
for x in (','.join(df.iloc[:, 3:7].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1).tolist())).split(','):
    disciplinas.add(x.strip()[3:])
disciplinas = list(disciplinas)

""" 
A - Conjunto de monitores
Candidadtos: são representados pelo número USP do aluno.
Ex: candidatos = [35543675, (...)]
"""
candidatos = df.iloc[0:,0].tolist()

""" 
Preferências: foram selecionadas pela ordem de preferência e depois os dados foram na forma de uma
função exponencial de modo a representar melhor os resultados finais. De qualquer forma, esse modelo 
pode ser alterado para ser ponderado pela nota para diferenciar candidatos. Assim,
o valor a ser colocado no lugar da preferência poderia ser um valor do interesse.

preferencias = { int(<nUSP>) : { string(<TURMA-1>):float(<INTERESSE>) }}
"""
preferencias = {}
for row in df.itertuples():
    opcoes = {}
    for col in [4, 5, 6, 7]:
        if row[col] is not np.nan:
            for i in row[col].split(','):
                opcoes[i.strip()[3:]] = row[9]
    preferencias[row[1]] = opcoes

# Existem valores não numéricos nas notas. Precisam ser tratados.
for n in preferencias:
    for d in preferencias[n]:
        if preferencias[n][d] == 'PÓS' or preferencias[n][d] == 'POS':
            preferencias[n][d] = 10.0

'''
N_a - Nota media de cada monitor
'''
N_a = {}
for row in df.itertuples():
    nusp = row[1]
    nota = row[9]
    if nota == 'PÓS':
        N_a[nusp] = 10.0
    else:
        try:
            N_a[nusp] = float(nota)
        except ValueError:
            N_a[nusp] = 0.0  # 0 se não for possivel converter


'''
s_ad - Preferencia dos monitores
'''
s_ad = {}
for a in candidatos:
    for d in disciplinas:
        if a in preferencias and d in preferencias[a]:
            s_ad[(a, d)] = 1
        else:
            s_ad[(a, d)] = 0

# Transformei em set para evitar candidatos e disciplinas duplicados
candidatos = list(set(candidatos))
disciplinas = list(set(disciplinas))

print(f'preferencias = {preferencias}')
print('Parametros')
print(f'A = {candidatos}')
print(f'D = {disciplinas}')
print(f's_ad = {s_ad}')
print(f'N_aa = {N_a}')
print(f'preferencias = {preferencias}')

preferencias = {11780000: {'SCC0122 - Estrutura de Dados': 6.8, 'SCC0201 - Introdução a Ciências de Computação II (T2)': 6.8, 'SCC0502 - Algoritmos e Estruturas de Dados I': 6.8}, 14590001: {'SLC0602 - Geometria Analítica': 8.8, 'SMA0304 - Álgebra Linear': 8.8, 'SMA0353 - Cálculo I': 8.8, 'SMA0375 - Álgebra Linear': 8.8, 'SMA0804 - Álgebra Linear para Estatística': 8.8, 'SME0300 - Cálculo Numérico (T1)': 8.8, 'SME0300 - Cálculo Numérico (T2)': 8.8, 'SCC0122 - Estrutura de Dados': 8.8, 'SCC0201 - Introdução a Ciências de Computação II (T2)': 8.8, 'SCC0502 - Algoritmos e Estruturas de Dados I': 8.8, 'SSC0304 - Introdução à Programação para Engenharias (T1)': 8.8, 'SSC0603 - Estrutura de Dados I': 8.8}, 12730002: {'SCC0604 - Programação Orientada a Objetos': 6.9, 'SSC0603 - Estrutura de Dados I': 6.9, 'SSC0902 - Organização e Arquitetura de Computadores (T1)': 6.9}, 10740003: {'SCC0122 - Estrutura de Dados': 6.5, 'SCC0201 - Introdução a Ciências de Computação II (T2)': 6.5, 'SCC0275 - Int

In [ ]:
'''
Modelo de Otimizacao
'''
modelo = LpProblem("Alocacao_de_Monitores", LpMaximize)

# Variaveis de decisao
x_ad = LpVariable.dicts("x", [(a, d) for a in candidatos for d in disciplinas], cat=LpBinary)
y_d = LpVariable.dicts("y", disciplinas, cat=LpBinary)

# Funcao objetivo
modelo += lpSum(N_a[a] * x_ad[(a, d)] for a in candidatos for d in disciplinas) - lpSum(y_d[d] for d in disciplinas)

'''
Restrições
'''

# Cada disciplina deve ter no maximo um monitor ou nao ter monitor
for d in disciplinas:
    modelo += lpSum(x_ad[(a, d)] for a in candidatos) + y_d[d] == 1, f"Restricao_disciplina_{d}"

# Cada monitor pode ser alocado a no maximo uma disciplina
for a in candidatos:
    modelo += lpSum(x_ad[(a, d)] for d in disciplinas) <= 1, f"Restricao_monitor_{a}"

# Um monitor so pode ser alocado a uma disciplina se ele estiver disposto (s_{ad} = 1)
for a in candidatos:
    for d in disciplinas:
        modelo += x_ad[(a, d)] <= s_ad[(a, d)], f"Restricao_disposicao_{a}_{d}"

'''
Resultado
'''

solver = PULP_CBC_CMD(msg=True)
modelo.solve(solver)

print("Status da solução:", modelo.status, modelo.status)
print("Valor da função objetivo:", modelo.objective.value())

# Monitores alocados
alocacoes = []
for a in candidatos:
    for d in disciplinas:
        if x_ad[(a, d)].varValue == 1:
            alocacoes.append((a, d, N_a[a]))
print("\nMonitores Alocados:")
for aloc in alocacoes:
    print(f"Monitor {aloc[0]} (Nota: {aloc[2]}) alocado à disciplina {aloc[1]}")

# Disciplinas sem monitores
disciplinas_sem_monitor = [d for d in disciplinas if y_d[d].varValue == 1]
print("\nDisciplinas sem monitores:")
for d in disciplinas_sem_monitor:
    print(f"Disciplina {d} está sem monitor")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/greff/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b3c30b8d7fd24d588956864d0aab2f08-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/b3c30b8d7fd24d588956864d0aab2f08-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6276 COLUMNS
At line 42754 RHS
At line 49026 BOUNDS
At line 55201 ENDATA
Problem MODEL has 6271 rows, 6174 columns and 18396 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 497.1 - 0.01 seconds
Cgl0002I 5685 variables fixed
Cgl0004I processed model has 116 rows, 436 columns (436 integer (436 of which binary)) and 818 elements
Cutoff increment increased from 1e-05 to 0.0999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -497.1
Cbc0038I Before mini branch and bound, 435 integers at bound

In [48]:
'''
Função para aumentar os dados de alunos e matérias com base na escala fornecida.

Parâmetros:
- file_path (str): Caminho para o arquivo Excel com os dados originais.
- escala (float): Fator de aumento dos dados (e.g., 1.5 para aumentar em 50%).

Retorno: Retorna os dados aumentados (alunos, notas, disponibilidade dos alunos para disciplinas e a lista de matérias).
'''
def aumentar_dados(path, escala):
    # Lê os dados originais usando a função existente
    seed = 0

    # Se seed for 0 e escala for 1, retornar os dados originais
    if seed == 0 and escala == 1:
        return 

    # Define a seed para a geração de dados aleatórios
    if seed is not None:
        random.seed(seed)

    novos_alunos = {}
    novas_notas = []
    nova_s_ad = []
    usp_existentes = candidatos
    max_usp = max(usp_existentes)

    # Aumenta o número de materias em 1.12x com base na escala
    num_materias_originais = len(disciplinas)
    num_novas_materias = math.ceil(num_materias_originais * 1.12 * escala)

    # Novos índices para as matérias que serão adicionadas
    novas_materias = list(range(num_materias_originais, num_materias_originais + num_novas_materias))

    # Aumenta o numero de alunos conforme a escala
    total_alunos = len(candidatos)
    num_novos_alunos = math.ceil(total_alunos * (escala - 1))  # Cria novos alunos além dos originais

    # Cria os novos alunos
    for i in range(num_novos_alunos):
        # Novo Nusp
        max_usp += 1
        novo_usp = max_usp
        usp_existentes.append(novo_usp)

        # Gerar uma nova nota aleatória para o novo aluno
        nova_nota = random.uniform(0, 10)
        novas_notas.append(nova_nota)

        # Gerar matérias aleatórias (entre 2 e 4) para cada novo aluno
        num_materias_aluno = random.randint(2, 4)
        todas_materias = list(range(num_materias_originais)) + novas_materias  # Considera matérias antigas e novas
        materias_aluno = random.sample(todas_materias, num_materias_aluno)

        # Criar a matriz de disponibilidade (s_ad) para o novo aluno com as matérias selecionadas
        disponibilidade_aluno = [1 if materia in materias_aluno else 0 for materia in todas_materias]
        nova_s_ad.append(disponibilidade_aluno)

        novos_alunos[novo_usp] = []

    # Expande a matriz s_ad original para acomodar as novas matérias
    s_ad_expandidos = [row + [0] * len(novas_materias) for row in s_ad]
    s_ad_expandidos += nova_s_ad

    # Combinar os dados originais e os novos
    alunos_expandidos = {**candidatos, **novos_alunos}
    Na_expandidos = N_a + novas_notas

    return alunos_expandidos, Na_expandidos, s_ad_expandidos, todas_materias

### Testes Computacionais

1. Execute o modelo utilizando a instância ”Dados_monitores.xlsx” (disponibilizada na página da disciplina) e apresente o resultado por meio da função informacoes_monitores (fornecida no código disponível na página da disciplina). Analise a solução considerando o número de monitores alocados e as disciplinas sem alocação de monitores.

In [49]:
print(f'Numero de monitores alocados: {len(alocacoes)}')
print("\nDisciplinas sem monitores:")
for d in disciplinas_sem_monitor:
    print(f"Disciplina {d} está sem monitor")

print("\nMonitores dispostos a monitorar disciplinas sem monitores:")
for d in disciplinas_sem_monitor:
    monitores_disponiveis = []
    for a in candidatos:
        if s_ad[(a, d)] == 1:
            alocado = any(x_ad[(a, d_prime)].varValue == 1 for d_prime in disciplinas)
            if not alocado:
                monitores_disponiveis.append((a, N_a[a]))
    if monitores_disponiveis:
        print(f"Disciplina {d.replace('_', ' ')}:")
        for monitor in monitores_disponiveis:
            print(f"  Monitor {monitor[0]} (Nota: {monitor[1]}) está disposto a monitorar.")
    else:
        print(f"Disciplina {d.replace('_', ' ')} não tem monitores disponíveis.")

Numero de monitores alocados: 59

Disciplinas sem monitores:
Disciplina SCC0205 - Teoria da Computação e Linguagens Formais (T2) está sem monitor
Disciplina SCC0213 - Metodologia de Pesquisa em Computação está sem monitor
Disciplina SME0822 - Análise Multivariada e Aprendizado Não Supervisionado está sem monitor
Disciplina SCC0250 - Computação Gráfica está sem monitor

Monitores dispostos a monitorar disciplinas sem monitores:
Disciplina SCC0205 - Teoria da Computação e Linguagens Formais (T2) não tem monitores disponíveis.
Disciplina SCC0213 - Metodologia de Pesquisa em Computação não tem monitores disponíveis.
Disciplina SME0822 - Análise Multivariada e Aprendizado Não Supervisionado não tem monitores disponíveis.
Disciplina SCC0250 - Computação Gráfica não tem monitores disponíveis.


As disciplinas ficaram sem monitores pois não há monitor que se interesse por elas.

2. Analise, do ponto de vista do gestor, o que fazer no caso de haver disciplinas sem monitores, mas existirem monitores que já cursaram essas disciplinas, porém não se inscreveram nelas. Não é necessário implementar nenhum código.

O ideal seria oferecer aos monitores que já cursaram a disciplina e obtiveram notas boas a possibilidade de monitorá-la.

3. Selecione arbitrariamente 5 alunos monitores e atribua-os a disciplinas compatíveis (dica: fixe as variáveis escolhidas em valores determinados). Exiba novamente o resultado gerado pelo modelo e comente brevemente as diferenças entre a nova solução e a anterior. A função objetivo foi significativamente alterada? O número de disciplinas não atendidas aumentou?

In [50]:
monitores_selecionados = random.sample(candidatos, 5)

monitorias_novas = []
for a in monitores_selecionados:

    disciplinas_preferidas = [d for d in disciplinas if s_ad[(a, d)] == 1]
    if disciplinas_preferidas:

        d = disciplinas_preferidas[0]
        monitorias_novas.append((a, d))
    else:
        print(f"Monitor {a} does not have any willing disciplines.")

for (a, d) in monitorias_novas:
    modelo += x_ad[(a, d)] == 1, f"Fixacao_{a}_{d}"

solver = PULP_CBC_CMD(msg=True)
modelo.solve(solver)

print("Status da solução:", modelo.status, modelo.status)
print("Valor da função objetivo:", modelo.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/greff/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/8537641aab854eeda863041d4a2db922-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/8537641aab854eeda863041d4a2db922-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6281 COLUMNS
At line 42764 RHS
At line 49041 BOUNDS
At line 55216 ENDATA
Problem MODEL has 6276 rows, 6174 columns and 18401 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.16 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.18   (Wallclock seconds):       0.21

Status da solução: -1 -1
Valor da função objetivo: 483.5000000000001


In [51]:
print(f'Numero de monitores alocados: {len(alocacoes)}')
print("\nDisciplinas sem monitores:")
for d in disciplinas_sem_monitor:
    print(f"Disciplina {d} está sem monitor")

Numero de monitores alocados: 59

Disciplinas sem monitores:
Disciplina SCC0205 - Teoria da Computação e Linguagens Formais (T2) está sem monitor
Disciplina SCC0213 - Metodologia de Pesquisa em Computação está sem monitor
Disciplina SME0822 - Análise Multivariada e Aprendizado Não Supervisionado está sem monitor
Disciplina SCC0250 - Computação Gráfica está sem monitor


4. Utilize a função aumentar_dados (fornecida no código disponível na página da disciplina) para gerar instâncias de maior complexidade. Crie 5 instâncias com fatores de escala 2, 3, 5, 10, 15. Apresente uma tabela com os resultados obtidos, incluindo o gap e o tempo de execução de cada instância. Comente sobre os resultados, estabelecendo um limite de 30 minutos para a resolução de cada instância.

In [52]:
alunos_expandidos, Na_expandidos, s_ad_expandidos, todas_materias = aumentar_dados('/home/greff/Desktop/scheduler-class-assistant/dataset/Dados_monitores.xlsx', escala=2)

print(type(alunos_expandidos))
print(type(Na_expandidos))
print(type(s_ad_expandidos))
print(type(todas_materias))

TypeError: can only concatenate tuple (not "list") to tuple